In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using Theano backend.


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

In [4]:
# Normalize the data
X_train = X_train / 255.0
test = test / 255.0
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,1,28,28)
test = test.values.reshape(-1,1,28,28)

In [5]:
Y_train = to_categorical(Y_train, num_classes = 10)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.05, random_state=2333)
print(X_train.shape, Y_train.shape)

(39900, 1, 28, 28) (39900, 10)


In [6]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (1,28,28)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
# Compile the model

opt = Adam(lr=0.001)
model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
print("compile done")

compile done


In [7]:
# With data augmentation to prevent overfitting (accuracy 0.99286)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # Randomly zoom image 
        width_shift_range=0.05,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

/home/jac/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3 or 4 channels on axis 1. However, it was passed an array with shape (39900, 1, 28, 28) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [8]:
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.models import load_model
best_model_path = 'best_m.h5'
best_model = ModelCheckpoint(filepath=best_model_path, monitor='val_loss', save_best_only=True)
epochs = 100
steps_per_epoch = 50
batch_size = 64
model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=steps_per_epoch,
                              callbacks=[best_model])
model = load_model(best_model_path)

Epoch 1/100
25s - loss: 1.8113 - acc: 0.3553 - val_loss: 0.8191 - val_acc: 0.7810
Epoch 2/100
26s - loss: 0.9103 - acc: 0.7044 - val_loss: 0.3159 - val_acc: 0.9052
Epoch 3/100
27s - loss: 0.5929 - acc: 0.8103 - val_loss: 0.1825 - val_acc: 0.9495
Epoch 4/100
25s - loss: 0.4479 - acc: 0.8469 - val_loss: 0.1354 - val_acc: 0.9590
Epoch 5/100
25s - loss: 0.3803 - acc: 0.8787 - val_loss: 0.1132 - val_acc: 0.9638
Epoch 6/100
25s - loss: 0.3734 - acc: 0.8812 - val_loss: 0.1020 - val_acc: 0.9719
Epoch 7/100
26s - loss: 0.3052 - acc: 0.9050 - val_loss: 0.1042 - val_acc: 0.9676
Epoch 8/100
27s - loss: 0.2944 - acc: 0.9056 - val_loss: 0.0838 - val_acc: 0.9724
Epoch 9/100
31s - loss: 0.3108 - acc: 0.9087 - val_loss: 0.0816 - val_acc: 0.9767
Epoch 10/100
27s - loss: 0.2569 - acc: 0.9231 - val_loss: 0.0685 - val_acc: 0.9795
Epoch 11/100
27s - loss: 0.2595 - acc: 0.9216 - val_loss: 0.0737 - val_acc: 0.9776
Epoch 12/100
26s - loss: 0.2302 - acc: 0.9269 - val_loss: 0.0671 - val_acc: 0.9800
Epoch 13/100


In [13]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [14]:
import arrow
ts = str(arrow.now())[:-10].replace(':','_').replace('.',"_")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen_{}.csv".format(ts),index=False)

print('save done')

save done
